In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Cel: Przygotowanie danych dotyczących migracji
* Przygotowanie datasetu używanego do analiz korelacji pewnych współczynników z liczbą migrantów:
    * Porównanie informacji POPULATION BY CITIZENSHIP, ALL VALID PERMITS, MIGRATION BY CITIZENSHIP
    * Dodanie do wyjściowego datasetu informacji na temat Ukraińskich uchodźcach
    * Dodanie informacji dotyczących pobytów tymczasowych

# POPULATION BY CITIZENSHIP

selected_countries = ["Poland", "Bulgaria", "Romania", "Hungary", "France", "Italy", "Germany", "Sweden", "Spain", "Greece", "Slovenia"]


Skondensowana tabela przedstawia dane dotyczące ilości populacji w wybranych krajach w następującym formacie:
* NAZWA KRAJU 
* ROK 
* LICZBA OBYWATELI Z UE 
* LICZBA OBYWATELI Z DANEGO KRAJU
* ŁĄCZNA LICZBA OBYWATELI OBCOKRAJOWCÓW

In [93]:
pop_cit = pd.read_csv('../processed_data/population_by_citizenship.csv')

In [56]:
pop_cit.head()

,citizen,geo,year,number of citizens in country
0,Andorra,Austria,2015,4
1,Andorra,Austria,2016,3
2,Andorra,Austria,2017,4
3,Andorra,Austria,2018,5
4,Andorra,Austria,2019,4


In [57]:
selected_countries = ["Poland", "Bulgaria", "Romania", "Hungary", "France", "Italy", "Germany", "Sweden", "Spain", "Greece", "Slovenia"]
pop_cit = pop_cit[pop_cit['geo'].isin(selected_countries)]

In [58]:
pop_cit.head()

,citizen,geo,year,number of citizens in country
20,Andorra,Bulgaria,2023,0
21,Andorra,Bulgaria,2024,0
39,Andorra,Germany,2015,30
40,Andorra,Germany,2016,38
41,Andorra,Germany,2017,40


In [59]:
eu_countries = [
    "Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece",
    "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg",
    "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia",
    "Slovenia", "Spain", "Sweden"
]

In [60]:
pop_cit = pop_cit[
    ~pop_cit["geo"].str.contains("European Union|Total", case=False, na=False) &
    ~pop_cit["citizen"].str.contains("Asia|America|Africa|Europe|Oceanian|Candidate|European Union|Unknown|EU|European|Total", case=False, na=False)
]

In [61]:
def classify_citizen(row):
    if row['citizen'] == 'Reporting country':
        return 'national'
    elif row['citizen'] in eu_countries:
        return 'EU'
    else:
        return 'non_EU'
    
pop_cit['citizen_type'] = pop_cit.apply(classify_citizen, axis=1)

In [62]:
summary = pop_cit.groupby(['geo', 'year', 'citizen_type'])['number of citizens in country'].sum().reset_index()

In [65]:
pivot = summary.pivot_table(
    index=['geo', 'year'],
    columns='citizen_type',
    values='number of citizens in country',
    fill_value=0
).reset_index()

In [ ]:
pivot[pivot['geo'] == 'Poland']  # checking the pivot table for Poland

citizen_type,geo,year,EU,national,non_EU
60,Poland,2015,0.0,37891051.0,110001.0
61,Poland,2016,37834800.0,37811676.0,152122.0
62,Poland,2017,37783928.0,37756724.0,213118.0
63,Poland,2018,37759287.0,37731606.0,242205.0
64,Poland,2019,37712047.0,37683061.0,293009.0
65,Poland,2020,37633096.0,37599899.0,362239.0
66,Poland,2021,0.0,36623052.0,448078.0
67,Poland,2022,0.0,36445251.0,442296.0
68,Poland,2023,0.0,36314890.0,436595.0
69,Poland,2024,0.0,36187908.0,430864.0


In [68]:
pivot['foreigners'] = pivot['non_EU'] + pivot['EU']

In [69]:
pivot.head()

citizen_type,geo,year,EU,national,non_EU,foreigners
0,Bulgaria,2015,0.0,6963308.0,63879.0,63879.0
1,Bulgaria,2016,0.0,6863112.0,71491.0,71491.0
2,Bulgaria,2017,0.0,6767835.0,76562.0,76562.0
3,Bulgaria,2018,0.0,6668711.0,82370.0,82370.0
4,Bulgaria,2019,0.0,6565004.0,91243.0,91243.0


In [71]:
pivot.to_csv('../processed_data/population_by_citizenship_combined_data.csv', index=False)

# MIGRRANTS BY CITIZENSHIP

selected_countries = ["Poland", "Bulgaria", "Romania", "Hungary", "France", "Italy", "Germany", "Sweden", "Spain", "Greece", "Slovenia"]

Uchodźcy wojenni z Ukrainy dla Szwecji i Polski nie są wliczani -> wliczono ich na podstawie zbioru danych UKRAINE TEMPORARY PROTECTION
Osoby ubiegające się o azyl, które regularnie zamieszkują w danym miejscu przez co najmniej 12 miesięcy dla Bułgarii, Węgier, Polski, Rumuni, Szwecji -> wliczono ich na podstawie zbioru danych XYZ

Skondensowana tabela przedstawia dane dotyczące ilości populacji w wybranych krajach w następującym formacie:
* geo - NAZWA KRAJU
* year - ROK
* EU - LICZBA IMIGRANTÓW Z UE
* Ukraine - LICZBA MIGRANTÓW Z UKRAINY
* national - LICZBA IMMIGRANTÓW Z TEGO KRAJU (obywatele powracający z emigracji)
* non_EU - LICZBA IMIGRANTÓW SPOZA UE
* foreigners - LICZBA IMIGRANTÓW OBCOKRAJOWCÓW -> UE + NON UE
* foreigners_pop_share- PROCENT JAKI STANOWIĄ IMMIGRANCI ZAGRANICZNI PRZYJĘCI W DANYM ROKU W CAŁEJ POPULACJI
* non_EU_pop_share - PROCENT JAKI STANOWIĄ IMMIGRANCI SPOZA UE PRZYJĘCI W DANYM ROKU W CAŁEJ POPULACJI
* ukraine_pop_share - PROCENT JAKI STANOWIĄ UKRAIŃSCY IMMIGRANCJI PRZYJĘCI W DANYM ROKU W CAŁEJ POPULACJI
* ukraine_for_share - PROCENT JAKI STANOWIĄ UKRAIŃSCY IMMIGRANCI PRZYJĘCI W DANYM ROKU WŚRÓD WSZYSTKICH OBCOKRAJOWCÓW
* non_EU_for_share - PROCENT JAKI STANOWIĄ IMMIGRANCI SPOZA EU WŚRÓD WSZYSTKICH OBCOKRAJOWCÓW (trzeba uważać z tym, bo dane dotyczące imigrantów z EU wydają się wątpliwe)

In [332]:
migr_cit = pd.read_csv('../processed_data/immigration_by_citizenship.csv')
migr_cit = migr_cit[migr_cit['geo'] != migr_cit['Country of citizenship']]

In [333]:
migr_cit.head()

,Country of citizenship,geo,year,Migrants number
0,Andorra,Austria,1998,1.0
1,Andorra,Austria,1999,1.0
2,Andorra,Austria,2001,0.0
3,Andorra,Austria,2002,0.0
4,Andorra,Austria,2003,0.0


In [334]:
selected_countries = ["Poland", "Bulgaria", "Romania", "Hungary", "France", "Italy", "Germany", "Sweden", "Spain", "Greece", "Slovenia"]
migr_cit = migr_cit[migr_cit['geo'].isin(selected_countries)]

In [335]:
eu_countries = [
    "Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece",
    "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg",
    "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia",
    "Slovenia", "Spain", "Sweden"
]

migr_cit = migr_cit[
    ~migr_cit["Country of citizenship"].str.contains("Asia|America|Africa|Europe|Oceanian|Candidate|European Union|Unknown|EU|European|Total", case=False, na=False)
]

In [336]:
def classify_citizen(row):
    if row['Country of citizenship'] == 'Ukraine':
        return 'Ukraine'
    if row['Country of citizenship'] == 'Reporting country':
        return 'national'
    elif row['Country of citizenship'] in eu_countries:
        return 'EU'
    else:
        return 'non_EU'
    
migr_cit.loc[:, 'citizen_type'] = migr_cit.apply(classify_citizen, axis=1)

summary = migr_cit.groupby(['geo', 'year', 'citizen_type'])['Migrants number'].sum().reset_index()

pivot = summary.pivot_table(
    index=['geo', 'year'],
    columns='citizen_type',
    values='Migrants number',
    fill_value=0
).reset_index()

In [337]:
pivot.head()

citizen_type,geo,year,EU,Ukraine,national,non_EU
0,Bulgaria,2007,6.0,1.0,1498.0,117.0
1,Bulgaria,2012,3293.0,348.0,4964.0,14632.0
2,Bulgaria,2013,1287.0,583.0,4682.0,25767.0
3,Bulgaria,2014,1160.0,665.0,9502.0,32277.0
4,Bulgaria,2015,1136.0,837.0,10722.0,26825.0


In [338]:
pivot['non_EU'] = pivot['non_EU'] + pivot['Ukraine']

In [339]:
pivot['foreigners'] = pivot['non_EU'] + pivot['EU']

In [340]:
pivot[pivot['geo'] == 'Germany']

citizen_type,geo,year,EU,Ukraine,national,non_EU,foreigners
31,Germany,1998,257616.0,14121.0,196956.0,942912.0,1200528.0
32,Germany,1999,275510.0,15285.0,200150.0,1066385.0,1341895.0
33,Germany,2000,0.0,0.0,191909.0,649249.0,649249.0
34,Germany,2001,277871.0,20307.0,193958.0,1117868.0,1395739.0
35,Germany,2002,273176.0,20578.0,184202.0,984197.0,1257373.0
36,Germany,2003,268555.0,17696.0,167216.0,930517.0,1199072.0
37,Germany,2004,303679.0,15000.0,177993.0,897105.0,1200784.0
38,Germany,2005,319785.0,10881.0,128051.0,837384.0,1157169.0
39,Germany,2006,321409.0,7514.0,103388.0,782635.0,1104044.0
40,Germany,2007,344689.0,7551.0,106014.0,804038.0,1148727.0


### Dodanie uchodźców wojennych z Ukrainy do danych o imigracji dla Polski i Szwecji

In [341]:
ukr = pd.read_csv('../processed_data/ukraine_temporary_protection.csv')

In [342]:
selected_countries = ['Poland', 'Sweden']

In [343]:
ukr = ukr[ukr['geo'].isin(selected_countries)]

In [344]:
ukr.head()

,Country of citizenship,geo,OBS_VALUE,month
913,Ukraine,Poland,675085.0,2022-03
914,Ukraine,Poland,1046815.0,2022-04
915,Ukraine,Poland,1142375.0,2022-05
916,Ukraine,Poland,1202190.0,2022-06
917,Ukraine,Poland,1258235.0,2022-07


In [345]:
ukr['month'] = pd.to_datetime(ukr['month'], format='%Y-%m')

In [346]:
ukr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 913 to 1057
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Country of citizenship  74 non-null     object        
 1   geo                     74 non-null     object        
 2   OBS_VALUE               74 non-null     float64       
 3   month                   74 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.9+ KB


In [347]:
ukr_dec = ukr[ukr['month'].dt.month == 12]

In [348]:
ukr_dec['year'] = ukr_dec['month'].dt.year

C:\Users\natal\AppData\Local\Temp\ipykernel_18548\1327383742.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukr_dec['year'] = ukr_dec['month'].dt.year


In [349]:
ukr_dec = ukr_dec[ukr_dec['year']==2022]

In [350]:
df_merged = pd.merge(pivot, ukr_dec, on=['geo', 'year'], how='left')

In [351]:
df_merged.head()

,geo,year,EU,Ukraine,national,non_EU,foreigners,Country of citizenship,OBS_VALUE,month
0,Bulgaria,2007,6.0,1.0,1498.0,118.0,124.0,NaN,NaN,NaT
1,Bulgaria,2012,3293.0,348.0,4964.0,14980.0,18273.0,NaN,NaN,NaT
2,Bulgaria,2013,1287.0,583.0,4682.0,26350.0,27637.0,NaN,NaN,NaT
3,Bulgaria,2014,1160.0,665.0,9502.0,32942.0,34102.0,NaN,NaN,NaT
4,Bulgaria,2015,1136.0,837.0,10722.0,27662.0,28798.0,NaN,NaN,NaT


In [352]:
df_merged['Ukraine'] = df_merged['Ukraine'].fillna(0) + df_merged['OBS_VALUE'].fillna(0)
df_merged['non_EU'] = df_merged['non_EU'].fillna(0) + df_merged['OBS_VALUE'].fillna(0)
df_merged['foreigners'] = df_merged['foreigners'].fillna(0) + df_merged['OBS_VALUE'].fillna(0)

In [353]:
df_merged = df_merged.drop(columns=['OBS_VALUE'])

In [354]:
df_merged.drop(columns=['month', 'Country of citizenship'], inplace=True)

In [355]:
df_merged[df_merged['geo'] == 'Poland']

,geo,year,EU,Ukraine,national,non_EU,foreigners
115,Poland,1999,0.0,0.0,7052.0,473.0,473.0
116,Poland,2000,0.0,0.0,6921.0,410.0,410.0
117,Poland,2001,0.0,0.0,6270.0,355.0,355.0
118,Poland,2002,0.0,0.0,6328.0,259.0,259.0
119,Poland,2003,0.0,0.0,6548.0,500.0,500.0
120,Poland,2004,0.0,0.0,8253.0,1242.0,1242.0
121,Poland,2005,0.0,0.0,8228.0,1136.0,1136.0
122,Poland,2006,367.0,609.0,8978.0,3251.0,3618.0
123,Poland,2007,185.0,678.0,13384.0,3029.0,3214.0
124,Poland,2009,0.0,0.0,142348.0,46730.0,46730.0


### Osoby ubiegające się o azyl -> dodanie danych dla: Bułgaria, Węgry, Polska, Rumunia, Szwecja

In [356]:
as_df = pd.read_csv('../processed_data/asylum_applicants.csv')

In [357]:
selected_countries = ['Poland', 'Sweden', 'Hungary', 'Bulgaria', 'Romania']

In [358]:
as_df = as_df[(as_df['geo'].isin(selected_countries)) & (as_df['Country of citizenship'] == 'Total')]

In [359]:
df_merged2 = pd.merge(df_merged, as_df, on=['geo', 'year'], how='left')

In [360]:
df_merged2.head()

,geo,year,EU,Ukraine,national,non_EU,foreigners,Country of citizenship,Number
0,Bulgaria,2007,6.0,1.0,1498.0,118.0,124.0,NaN,NaN
1,Bulgaria,2012,3293.0,348.0,4964.0,14980.0,18273.0,Total,1385.0
2,Bulgaria,2013,1287.0,583.0,4682.0,26350.0,27637.0,Total,7145.0
3,Bulgaria,2014,1160.0,665.0,9502.0,32942.0,34102.0,Total,11080.0
4,Bulgaria,2015,1136.0,837.0,10722.0,27662.0,28798.0,Total,20390.0


In [361]:
df_merged2['non_EU'] = df_merged2['non_EU'].fillna(0) + df_merged2['Number'].fillna(0)
df_merged2['foreigners'] = df_merged2['foreigners'].fillna(0) + df_merged2['Number'].fillna(0)

In [362]:
df_merged2.drop(columns=['Number', 'Country of citizenship'], inplace=True)

### Jaki procent populacji to przyjęci migranci?

In [363]:
pop_cit = pd.read_csv('../processed_data/country_population.csv')

In [364]:
selected_countries = ["Poland", "Bulgaria", "Romania", "Hungary", "France", "Italy", "Germany", "Sweden", "Spain", "Greece", "Slovenia"]
pop_cit = pop_cit[(pop_cit['geo'].isin(selected_countries)) ]

In [365]:
pop_cit.head()

,geo,year,Population number
146,Bulgaria,2003,7805506
147,Bulgaria,2004,7745147
148,Bulgaria,2005,7688573
149,Bulgaria,2006,7629371
150,Bulgaria,2007,7572673


In [366]:
merged3 = pd.merge(df_merged2, pop_cit, on=['geo', 'year'], how='left')

In [367]:
merged3.head()

,geo,year,EU,Ukraine,national,non_EU,foreigners,Population number
0,Bulgaria,2007,6.0,1.0,1498.0,118.0,124.0,7572673.0
1,Bulgaria,2012,3293.0,348.0,4964.0,16365.0,19658.0,7327224.0
2,Bulgaria,2013,1287.0,583.0,4682.0,33495.0,34782.0,7202556.0
3,Bulgaria,2014,1160.0,665.0,9502.0,44022.0,45182.0,7117453.0
4,Bulgaria,2015,1136.0,837.0,10722.0,48052.0,49188.0,7029690.0


In [368]:
merged3['foreigners_pop_share'] = merged3['foreigners'] / merged3['Population number'] * 100
merged3['non_EU_pop_share'] = merged3['non_EU'] / merged3['Population number'] * 100
merged3['ukraine_pop_share'] = merged3['Ukraine'] / merged3['Population number'] * 100
merged3['ukraine_for_share'] = merged3['Ukraine'] / merged3['foreigners'] * 100
merged3['non_EU_for_share'] = merged3['non_EU'] / merged3['foreigners'] * 100
merged3.drop(columns=['Population number'], inplace=True)

In [369]:
merged3[merged3['geo']=='Poland']

,geo,year,EU,Ukraine,national,non_EU,foreigners,foreigners_pop_share,non_EU_pop_share,ukraine_pop_share,ukraine_for_share,non_EU_for_share
115,Poland,1999,0.0,0.0,7052.0,473.0,473.0,NaN,NaN,NaN,0.000000,100.000000
116,Poland,2000,0.0,0.0,6921.0,410.0,410.0,NaN,NaN,NaN,0.000000,100.000000
117,Poland,2001,0.0,0.0,6270.0,355.0,355.0,NaN,NaN,NaN,0.000000,100.000000
118,Poland,2002,0.0,0.0,6328.0,259.0,259.0,NaN,NaN,NaN,0.000000,100.000000
119,Poland,2003,0.0,0.0,6548.0,500.0,500.0,0.001308,0.001308,0.000000,0.000000,100.000000
120,Poland,2004,0.0,0.0,8253.0,1242.0,1242.0,0.003252,0.003252,0.000000,0.000000,100.000000
121,Poland,2005,0.0,0.0,8228.0,1136.0,1136.0,0.002976,0.002976,0.000000,0.000000,100.000000
122,Poland,2006,367.0,609.0,8978.0,3251.0,3618.0,0.009482,0.008520,0.001596,16.832504,89.856274
123,Poland,2007,185.0,678.0,13384.0,3029.0,3214.0,0.008430,0.007945,0.001778,21.095208,94.243933
124,Poland,2009,0.0,0.0,142348.0,57325.0,57325.0,0.150318,0.150318,0.000000,0.000000,100.000000


In [370]:
merged3.to_csv('../processed_data/immigration_by_citizenship_combined_data.csv', index=False)